In [ ]:
!pip install mlxtend
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
import psycopg2
con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
port= '5439', user= 'mantix', password= 'Mantix123!')
cur = con.cursor()

In [ ]:
# cur.execute("select * from cdp.orders_data;")
# orders_data = pd.read_sql("select * from cdp.orders_data;",con)

In [ ]:
# orders_data['order_number']=orders_data['order_number'].astype(int).astype(str)

In [ ]:
cur.execute("select * from cdp.products_data;")
products_data = pd.read_sql("select * from cdp.products_data;",con)

In [ ]:
products_data.columns

In [ ]:
products_data.shape

In [ ]:
prod_url = products_data[['product_name','image_url','product_code','product_description']].drop_duplicates()
print(prod_url.shape)
print(prod_url.isna().sum())
# prod_url=dict(zip(prod_url.product_name,prod_url.image_url))

In [ ]:
prod_url.head(2)

In [ ]:
import re
prod_url['product_description']=prod_url['product_description'].apply(lambda x:re.sub('[^A-Za-z]+', ' ', x))

In [ ]:
prod_url = prod_url[prod_url['product_description'].notnull() & (prod_url['product_description'].astype(str).str.strip() != "")]

In [ ]:
prod_url.shape

In [ ]:
# prod_url.dropna(subset=['product_description'], inplace=True)

In [ ]:
# prod_url = prod_url[prod_url['product_description'] != " "]

In [ ]:
# is_null_or_empty = prod_url['product_description'].isnull().values.any() or prod_url['product_description'].astype(str).str.strip().empty

In [ ]:
# selected_records = prod_url['product_description'].loc[prod_url['product_description'].str.contains('"')]

In [ ]:
prod_url.dtypes

In [ ]:
# Prod_url_copy=prod_url.copy()

In [ ]:
# Prod_url_copy.shape

In [ ]:
import re
prod_url['product_name']=prod_url['product_name'].apply(lambda x:re.sub('[^A-Za-z]+', ' ', x))

In [ ]:
prod_url.drop_duplicates(subset='product_code',keep='first',inplace=True)

In [ ]:
prod_url.shape

In [ ]:
prod_list=prod_url['product_name'].tolist()

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
#Encoding:
sen_embeddings = model.encode(prod_list)

In [ ]:
sen_embeddings.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#let's calculate cosine similarity for sentence 0:
cos_0=cosine_similarity(
    sen_embeddings,sen_embeddings
)

In [ ]:
cos_0

In [ ]:
cos_0=pd.DataFrame(cos_0)

In [ ]:
# cos_0.columns=prod_list

In [ ]:
prod_list_code=prod_url['product_code'].tolist()

In [ ]:
cos_0.columns=prod_list_code

In [ ]:
cos_0.index=prod_url['product_name']

In [ ]:
# cos=cos_0.round(2)

In [ ]:
cos_0.head(2)

In [ ]:
cos_0.iloc[1].sort_values(ascending=False)[1:4].astype(float)

In [ ]:
similarity_score={}
for i in range(len(cos_0)):
    # print(":::::::::::",i)
    similarity_score[cos_0.index.tolist()[i]]=cos_0.iloc[i].sort_values(ascending=False).astype(float).round(2)[1:4].to_dict()

In [ ]:
# similarity_score

In [ ]:
sim_score=pd.DataFrame.from_dict([similarity_score]).T

In [ ]:
sim_score.head(2)

In [ ]:
sim_score.reset_index(inplace=True)

In [ ]:
sim_score.head(2)

In [ ]:
sim_score.columns=['product_name','often_bundled_products']

In [ ]:
sim_score.head(2)

In [ ]:
final_df=pd.merge(prod_url,sim_score,how='left',on='product_name')

In [ ]:
final_df.head(2)

In [ ]:
final_df['tenant_id']='TNB00001'
final_df['store_id']='STOM000000001'

In [ ]:
from datetime import date
today_date = date.today()
final_df['create_dms'] = str(today_date)

In [ ]:
str(today_date)

In [ ]:
 str(final_df.loc[0]['often_bundled_products'])

In [ ]:
# final_df=final_df[:50]

In [ ]:
final_df.head(2)

In [ ]:
# final_df['often_bundled_products'].tolist()

In [ ]:
from itertools import islice

def dict_to_object(t):
    t1={}
    t2={}
    t3={}
    final_dict={}
    inc = iter(t.items())
    res1 = dict(islice(inc, len(t) // 3))
    res2 = dict(islice(inc, len(t) // 2))
    res3 = dict(inc)
    for i,j in res1.items():
        t1['productCode']=i
        t1['conf_score']=j
    for d,k in res2.items():
        t2['productCode']=d
        t2['conf_score']=k
    for x,y in res3.items():
        t3['productCode']=x
        t3['conf_score']=y
    final_dict['oftenBundled']=[t1,t2,t3]
    
    return final_dict

In [ ]:
final_df['often_bundled_products']=final_df['often_bundled_products'].apply(lambda x:dict_to_object(x))

In [ ]:
final_df['often_bundled_products'].tolist()

In [ ]:
final_df.head(2)

In [ ]:
final_df.loc[0]['product_code']

In [ ]:
# import json
# strjson.dumps(final_df.loc[0]['often_bundled_products'])

In [ ]:
from psycopg2.extras import execute_values
import json
import datetime
con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
port= '5439', user= 'mantix', password= 'Mantix123!')
cur = con.cursor()

def drop_table(conn,table):
    try:
        query = f"""DROP TABLE IF EXISTS {table}"""
        cur = conn.cursor()
        cur.execute(query)
        conn.commit()
        print(f'Drop table {table} successfully')
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        #return 1
#drop_table

def create_table(conn,table):
    try:
        query_col = """(
            product_url VARCHAR(1000),product_code VARCHAR(1000), often_bundled_products VARCHAR(1000),tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
        query = f"""
                CREATE TABLE {table} 
                """+query_col
        cur = conn.cursor()
        cur.execute("ROLLBACK")
        conn.commit()
        cur.execute(query)
        conn.commit()
        print(f'Successfully created the table {table}')
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        return 1
#create_table

def single_insert(conn, insert_req):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()
#single_insert

def batch_insert(conn, batch_df):
    # Create a cursor object from the connection
    cursor = conn.cursor()
    statment=f"INSERT into {table} (product_url, product_code, often_bundled_products, tenant_id, store_id, create_dms) values (%s, %s, %s, %s, %s, %s)"
    try:
        # Execute the batch INSERT request
        cursor.executemany(statment, batch_df)
        # Commit the changes to the database
        conn.commit()
        print("Batch INSERT successful.")
    except (Exception, psycopg2.DatabaseError) as error:
        # Rollback the transaction in case of an error
        conn.rollback()
        print("Batch INSERT failed, Error:", error)
    finally:
        # Close the cursor
        cursor.close()
#batch_insert

def insert_dataframe(final_df,conn,table):
    batch_df=[]
    for i in final_df.index:
        current_tuple=(
            str(final_df.loc[i]['image_url']), 
            str(final_df.loc[i]['product_code']), 
            str(json.dumps(final_df.loc[i]['often_bundled_products'])),
            str(final_df.loc[i]['tenant_id']),
            str(final_df.loc[i]['store_id']),
            str(final_df.loc[i]['create_dms']),
        )
        batch_df.append(current_tuple)
        if (len(batch_df) == 500): 
            batch_insert(conn, batch_df)
            batch_df=[]
            print("Current index: {}, time: {}".format(i, datetime.datetime.now()))
        #endif
    # Insert any remaining rows
    if (len(batch_df) > 0):
        batch_insert(conn,batch_df)
    print('Successfully inserted dataframe into the table')
#insert_dataframe

table = 'cdp.often_bundled_rules'
print("before drop")
drop_table(con,table)
print("before create")
create_table(con,table)
print("before insert")
startTime = datetime.datetime.now()
insert_dataframe(final_df,con,table)
print("Start: {}, Finish: {}".format(startTime, datetime.datetime.now()))

In [ ]:
# from psycopg2.extras import execute_values
# import json
# import datetime
# con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
# port= '5439', user= 'mantix', password= 'Mantix123!')
# cur = con.cursor()

# def drop_table(conn,table):
#     try:
#         query = f"""DROP TABLE IF EXISTS {table}"""
#         cur = conn.cursor()
#         cur.execute(query)
#         conn.commit()
#         print(f'Drop table {table} successfully')
#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         conn.rollback()
#         cur.close()
#         #return 1
# #drop_table

# def create_table(conn,table):
#     try:
#         query_col = """(
#             product_url VARCHAR(1000),product_code VARCHAR(1000), often_bundled_products VARCHAR(1000),tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
#         query = f"""
#                 CREATE TABLE {table} 
#                 """+query_col
#         cur = conn.cursor()
#         cur.execute("ROLLBACK")
# #         conn.commit()
#         cur.execute(query)
#         conn.commit()
#         print(f'Successfully created the table {table}')
#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         conn.rollback()
#         cur.close()
#         return 1
# #create_table

# def single_insert(conn, insert_req):
#     """ Execute a single INSERT request """
#     cursor = conn.cursor()
#     try:
#         cursor.execute(insert_req)
#         conn.commit()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         conn.rollback()
#         cursor.close()
#         return 1
#     cursor.close()
# #single_insert

# def batch_insert(conn, batch_df):
#     # Create a cursor object from the connection
#     cursor = conn.cursor()
#     statment=f"INSERT into {table} (product_url, product_code, often_bundled_products, tenant_id, store_id, create_dms) values (%s, %s, %s, %s, %s, %s)"
#     try:
#         # Execute the batch INSERT request
#         cursor.executemany(statment, batch_df)
#         # Commit the changes to the database
#         conn.commit()
#         print("Batch INSERT successful.")
#     except (Exception, psycopg2.DatabaseError) as error:
#         # Rollback the transaction in case of an error
#         conn.rollback()
#         print("Batch INSERT failed, Error:", error)
#     finally:
#         # Close the cursor
#         cursor.close()
# #batch_insert

# def insert_dataframe(final_df,conn,table):
#     batch_df=[]
#     for i in final_df.index:
#         current_tuple=(
#             "'"+str(final_df.loc[i]['image_url'])+"'", 
#             "'"+int(final_df.loc[i]['product_code'])+"'", 
#             "'"+json.dumps(final_df.loc[i]['often_bundled_products'])+"'",
#             "'"+int(final_df.loc[i]['tenant_id'])+"'",
#             "'"+int(final_df.loc[i]['store_id'])+"'",
#             "'"+int(final_df.loc[i]['create_dms'])+"'",
#         )
#         batch_df.append(current_tuple)
#         if (len(batch_df) == 500): 
#             batch_insert(conn, batch_df)
#             batch_df=[]
#             print("Current index: {}, time: {}".format(i, datetime.datetime.now()))
#         #endif
#     print('Successfully inserted dataframe into the table')
# #insert_dataframe

# table = 'cdp.often_bundled_rules'
# print("before drop")
# drop_table(con,table)
# print("before create")
# create_table(con,table)
# print("before insert")
# startTime = datetime.datetime.now()
# insert_dataframe(final_df,con,table)
# print("Start: {}, Finish: {}".format(startTime, datetime.datetime.now()))

In [ ]:
# from concurrent.futures import ThreadPoolExecutor

# from psycopg2.extras import execute_values
# import json
# import datetime
# con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cgzkthavydhk.us-east-2.redshift.amazonaws.com', 
# port= '5439', user= 'mantix', password= 'Mantix123!')
# cur = con.cursor()

# def drop_table(conn,table):
#     try:
#         query = f"""DROP TABLE IF EXISTS {table}"""
#         cur = conn.cursor()
#         cur.execute(query)
#         conn.commit()
#         print(f'Drop table {table} successfully')
#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         conn.rollback()
#         cur.close()
#         #return 1
# #drop_table

# def create_table(conn,table):
#     try:
#         query_col = """(
#             product_url VARCHAR(1000),product_code VARCHAR(1000), often_bundled_products VARCHAR(1000),tenant_id VARCHAR(1000),store_id VARCHAR(1000),create_dms VARCHAR(1000))"""
#         query = f"""
#                 CREATE TABLE {table} 
#                 """+query_col
#         cur = conn.cursor()
#         cur.execute("ROLLBACK")
#         conn.commit()
#         cur.execute(query)
#         conn.commit()
#         print(f'Successfully created the table {table}')
#         cur.close()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         conn.rollback()
#         cur.close()
#         return 1
# #create_table

# def single_insert(conn, insert_req):
#     """ Execute a single INSERT request """
#     cursor = conn.cursor()
#     try:
#         cursor.execute(insert_req)
#         conn.commit()
#     except (Exception, psycopg2.DatabaseError) as error:
#         print("Error: %s" % error)
#         conn.rollback()
#         cursor.close()
#         return 1
#     cursor.close()
# #single_insert

# def batch_insert(conn, table, batch_df):
#     # Create a cursor object from the connection
#     cursor = conn.cursor()
#     statement = f"INSERT into {table} (product_url, product_code, often_bundled_products, tenant_id, store_id, create_dms) values (%s, %s, %s, %s, %s, %s)"
#     try:
#         # Execute the batch INSERT request
#         cursor.executemany(statement, batch_df)
#         # Commit the changes to the database
#         conn.commit()
#         print("Batch INSERT successful.")
#     except (Exception, psycopg2.DatabaseError) as error:
#         # Rollback the transaction in case of an error
#         conn.rollback()
#         print("Batch INSERT failed, Error:", error)
#     finally:
#         # Close the cursor
#         cursor.close()

# def insert_dataframe(final_df, conn, table):
#     batch_df = []
#     insert_count = 0

#     def insert_batch(batch):
#         nonlocal insert_count
#         with conn.cursor() as cursor:
#             try:
#                 cursor.executemany(statement, batch)
#                 conn.commit()
#                 insert_count += len(batch)
#                 print("Batch INSERT successful. Inserted:", insert_count)
#             except (Exception, psycopg2.DatabaseError) as error:
#                 conn.rollback()
#                 print("Batch INSERT failed, Error:", error)

#     statement = f"INSERT into {table} (product_url, product_code, often_bundled_products, tenant_id, store_id, create_dms) values (%s, %s, %s, %s, %s, %s)"

#     with ThreadPoolExecutor() as executor:
#         for i in final_df.index:
#             current_tuple = (
#                 "'" + final_df.loc[i]['image_url'] + "'",
#                 "'" + final_df.loc[i]['product_code'] + "'",
#                 "'" + json.dumps(final_df.loc[i]['often_bundled_products']) + "'",
#                 "'" + final_df.loc[i]['tenant_id'] + "'",
#                 "'" + final_df.loc[i]['store_id'] + "'",
#                 "'" + final_df.loc[i]['create_dms'] + "'",
#             )
#             batch_df.append(current_tuple)
#             if len(batch_df) == 500:
#                 executor.submit(insert_batch, batch_df)
#                 batch_df = []
#                 print("Current index: {}, time: {}".format(i, datetime.datetime.now()))

#     # Insert any remaining rows
#     if batch_df:
#         insert_batch(batch_df)

#     print('Successfully inserted dataframe into the table')

# table = 'cdp.often_bundled_rules'
# print("before drop")
# drop_table(con,table)
# print("before create")
# create_table(con,table)
# print("before insert")
# startTime = datetime.datetime.now()
# insert_dataframe(final_df,con,table)
# print("Start: {}, Finish: {}".format(startTime, datetime.datetime.now()))

###### 